In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
# Standard library imports
import os
import re
import sys
import json
import base64
from io import BytesIO

# Other library imports
import boto3
import numpy as np
# import seaborn as sns  # Comment out this line temporarily
from PIL import Image
from scipy.spatial.distance import cdist

# Print SDK versions
print(f"Python version: {sys.version.split()[0]}")
print(f"Boto3 SDK version: {boto3.__version__}")

# If you need seaborn functionality, try importing specific modules
try:
    import seaborn as sns
    print("Seaborn imported successfully")
except ValueError as e:
    print(f"Seaborn import failed: {e}")
    print("You may need to update seaborn and numpy versions")